In [12]:
import cv2
import numpy as np

BYTEORDER = 'big'  # endianness is intra-byte (?)
# specs: https://qoiformat.org/qoi-specification.pdf

In [13]:
# Generate end marker
i0, i1 = 0, 1
b0, b1 = i0.to_bytes(1, byteorder=BYTEORDER, signed=False), i1.to_bytes(1, byteorder=BYTEORDER, signed=False)
end_marker = i0.to_bytes(7, byteorder=BYTEORDER, signed=False) + i1.to_bytes(1, byteorder=BYTEORDER, signed=False)

In [14]:
print(end_marker)  # b'\x00\x00\x00\x00\x00\x00\x00\x01'

b'\x00\x00\x00\x00\x00\x00\x00\x01'


In [15]:
def index_hash (pixel: np.ndarray):
    if pixel.shape[0] == 4:
        r, g, b, a = pixel
        index_position = (r * 3 + g * 5 + b * 7 + a * 11) % 64
    elif pixel.shape[0] == 3:
        r, g, b = pixel  # a = 0
        index_position = (r * 3 + g * 5 + b * 7) % 64
    else:
        print(pixel, pixel.shape)
        raise ValueError('Wrong pixel shape')
    return int(index_position)

In [16]:
path = 'src/IMG_0036-v3-duplicate-downres.jpeg'

img = cv2.imread(path)
height, width, channels = img.shape
colorspace = 0  # srgb

print(f'Loaded image {path} ({width}x{height}) with {channels} channels.')

if channels == 3:
    pixel_prev = np.array([0, 0, 0])
elif channels == 4:
    pixel_prev = np.array([0, 0, 0, 255])
else: print('error: wrong number of channels')

img_line = img.reshape((-1, 3))
print(f'Estimated processing time: {round(img_line.shape[0]/6000)}s')

Loaded image src/IMG_0036-v3-duplicate-downres.jpeg (600x400) with 3 channels.
Estimated processing time: 40s


## Encoding

In [17]:
# Generate header
width_b = width.to_bytes(4, byteorder=BYTEORDER, signed=False)
height_b = height.to_bytes(4, byteorder=BYTEORDER, signed=False)
channels_b = channels.to_bytes(1, byteorder=BYTEORDER, signed=False)
colorspace_b = colorspace.to_bytes(1, byteorder=BYTEORDER, signed=False)

header = b'qoif' + width_b + height_b + channels_b + colorspace_b

In [6]:
bytes = b''

# Initialise run length encoding buffer
run_len = 0
run = False

# Initialise memory
if channels == 3:
    memory = {i: np.array([0, 0, 0]) for i in range(64)}
elif channels == 4:
    memory = {i: np.array([0, 0, 0, 255]) for i in range(64)}

for pixel in img_line:

    # Unpack pixel
    if channels == 3:
        r, g, b = pixel
        r_prev, g_prev, b_prev = pixel_prev
        a, a_prev = 255, 255
    elif channels == 4:
        r, g, b, a = pixel
        r_prev, g_prev, b_prev, a_prev = pixel_prev


    # Run-length encoding
    if (pixel == pixel_prev).all() and run_len < 63:
        # print('run')
        run_len += 1
        run = True


    else:
        # End run-length
        if run == True:
            val = '11' + '{0:06b}'.format(run_len - 1)  # store with a bias of -1 (to make full use of the 6-bit 
            val_b = int(val, 2).to_bytes(1, byteorder=BYTEORDER, signed=False)
            bytes += val_b  # write
            run_len = 0
            run = False

        # Memory search
        flag = False
        for key in memory.keys():
            if (memory[key] == pixel).all():
                # print('from memory')
                val = '00' + '{0:06b}'.format(key)
                val_b = int(val, 2).to_bytes(1, byteorder=BYTEORDER, signed=False)
                flag = True
                break
        
        if flag == False:
            
            dr = (int(r) - int(r_prev)) % 256
            dg = (int(g) - int(g_prev)) % 256
            db = (int(b) - int(b_prev)) % 256
            da = (int(a) - int(a_prev)) % 256

            # Diff
            if dr in range(-2, 2) and dg in range(-2, 2) and db in range(-2, 2) and da == 0:
                # print('smalldif')
                val = '01' + '{0:02b}'.format(dr+2) + '{0:02b}'.format(dg+2) + '{0:02b}'.format(db+2)
                val_b = int(val, 2).to_bytes(1, byteorder=BYTEORDER, signed=False)
            
            # Luma
            else:
                dr_dg = (dr - dg) % 256
                db_dg = (db - dg) % 256
                
                if dg in range(-32, 32) and dr_dg in range(-8, 8) and db_dg in range(-8, 8) and da == 0:
                    # print('bigdif')
                    vals = ['10' + '{0:06b}'.format(dg+32), '{0:04b}'.format(dr_dg+8) + '{0:04b}'.format(db_dg+8)]
                    vals_b = [int(val, 2).to_bytes(1, byteorder=BYTEORDER, signed=False) for val in vals]
                    val_b = vals_b[0] + vals_b[1]
                
                # New pixel
                else:
                    # print('new')
                    if channels == 3 or (channels == 4 and da == 0):
                        vals = ['11111110', '{0:08b}'.format(int(r)), '{0:08b}'.format(int(g)), '{0:08b}'.format(int(b))]
                        vals_b = [int(val, 2).to_bytes(1, byteorder=BYTEORDER, signed=False) for val in vals]
                        val_b = vals_b[0] + vals_b[1] + vals_b[2] + vals_b[3]
                        # tag = i1.to_bytes(7, byteorder=BYTEORDER, signed=False) + i0.to_bytes(1, byteorder=BYTEORDER, signed=False)
                        # val = int(r).to_bytes(8, byteorder=BYTEORDER, signed=False) + int(g).to_bytes(8, byteorder=BYTEORDER, signed=False) + int(b).to_bytes(8, byteorder=BYTEORDER, signed=False)
                    elif channels == 4:
                        vals = ['11111111', '{0:08b}'.format(int(r)), '{0:08b}'.format(int(g)), '{0:08b}'.format(int(b)), '{0:08b}'.format(int(a))]
                        vals_b = [int(val, 2).to_bytes(1, byteorder=BYTEORDER, signed=False) for val in vals]
                        val_b = vals_b[0] + vals_b[1] + vals_b[2] + vals_b[3] + vals_b[4]
                        # tag = i1.to_bytes(8, byteorder=BYTEORDER, signed=False)
                        # val = int(r).to_bytes(8, byteorder=BYTEORDER, signed=False) + int(g).to_bytes(8, byteorder=BYTEORDER, signed=False) + int(b).to_bytes(8, byteorder=BYTEORDER, signed=False) + int(a).to_bytes(8, byteorder=BYTEORDER, signed=False)

        # Write
        bytes += val_b

    # Add pixel to memory
    memory[index_hash(pixel)] = pixel
    pixel_prev = pixel.copy()

In [7]:
with open('out.bin', 'wb') as out_bin:
    out_bin.write(header)
    out_bin.write(bytes)
    out_bin.write(end_marker)


## Decoding

In [19]:
with open('out.bin', 'rb') as in_bin:
    img_bytes = in_bin.read()

# with open('src/testcard.qoi', 'rb') as in_bin:
#     img_bytes = in_bin.read()

header = img_bytes[:14]
bytes = img_bytes[14:-8]
# end_marker = img_bytes[-8:]


tag = header[:4].decode('utf-8')
width = int.from_bytes(header[4:8], BYTEORDER)
height = int.from_bytes(header[8:12], BYTEORDER)
channels = header[12]
colorspace = header[12]


### TODO: READ NB OF CHANNELS

3


In [11]:
nbytes = len(bytes)

if channels == 3:
    img_line = np.empty((0, 3), dtype=np.uint8)
elif channels == 4:
    img_line = np.empty((0, 4), dtype=np.uint8)

if channels == 3:
    pixel_prev = np.array([0, 0, 0], dtype=np.uint8)
elif channels == 4:
    pixel_prev = np.array([0, 0, 0, 255], dtype=np.uint8)
else: print('error: wrong number of channels')

if channels == 3:
    memory = {i: np.array([0, 0, 0]) for i in range(64)}
elif channels == 4:
    memory = {i: np.array([0, 0, 0, 255]) for i in range(64)}

i = 0
while i < nbytes:

    byte = bytes[i]
    byte_bin = '{0:08b}'.format(byte)

    # Original value (RGB)
    if channels == 3 and byte_bin == '11111110':
        r = bytes[i+1]
        g = bytes[i+2]
        b = bytes[i+3]
        pixel = np.array([r, g, b], dtype=np.uint8)
        img_line = np.concatenate((img_line, np.expand_dims(pixel, axis=0)))
        
        memory[index_hash(pixel)] = pixel
        pixel_prev = pixel.copy()
        i += 4

    # Original value (RGBA)
    elif channels == 4 and byte_bin == '11111111':
        r = bytes[i+1]
        g = bytes[i+2]
        b = bytes[i+3]
        a = bytes[i+4]
        pixel = np.array([r, g, b, a], dtype=np.uint8)
        img_line = np.concatenate((img_line, np.expand_dims(pixel, axis=0)))
        
        memory[index_hash(pixel)] = pixel
        pixel_prev = pixel.copy()
        i += 5

    # Run
    elif byte_bin[:2] == '11':
        run_len = int(byte_bin[2:], 2)
        pixels = np.full((run_len, channels), pixel_prev, dtype=np.uint8)  # +1 here or encode run_len+1?
        img_line = np.concatenate((img_line, pixels))

        i += 1

    # Big dif
    elif byte_bin[:2] == '10':
        # print('bigdif')
        dg = int(byte_bin[2:], 2) - 32

        byte_next = bytes[i+1]
        byte_next_bin = '{0:08b}'.format(byte_next)
        dr = int(byte_next_bin[:4], 2) - 8 + dg
        db = int(byte_next_bin[4:], 2) - 8 + dg

        if channels == 3: pixel_diff = np.array([dr, dg, db])
        elif channels == 4: pixel_diff = np.array([dr, dg, db, 0])
        pixel = (pixel_prev.copy() + pixel_diff).astype(np.uint8)
        img_line = np.concatenate((img_line, np.expand_dims(pixel, axis=0)))
        
        memory[index_hash(pixel)] = pixel
        pixel_prev = pixel.copy()
        i += 2

    # Small dif
    elif byte_bin[:2] == '01':
        dr = int(byte_bin[2:4], 2) - 2
        dg = int(byte_bin[4:6], 2) - 2
        db = int(byte_bin[6:8], 2) - 2
        
        if channels == 3: pixel_diff = np.array([dr, dg, db])
        elif channels == 4: pixel_diff = np.array([dr, dg, db, 0])
        pixel = (pixel_prev.copy() + pixel_diff).astype(np.uint8)
        # memory[index_hash(pixel)] = pixel
        img_line = np.concatenate((img_line, np.expand_dims(pixel, axis=0)))

        memory[index_hash(pixel)] = pixel
        pixel_prev = pixel.copy()
        i += 1

    # Index
    elif byte_bin[:2] == '00':
        index = int(byte_bin[2:], 2)
        pixel = memory[index].copy()
        # memory[index_hash(pixel)] = pixel
        img_line = np.concatenate((img_line, np.expand_dims(pixel, axis=0)))

        pixel_prev = pixel.copy()
        i += 1


In [9]:
print(img_line.shape)

(240000, 3)


In [10]:

# cv2.imwrite('out.png', img_line.copy().reshape((256, 256, 4)))
cv2.imwrite('out.png', img_line.copy().reshape((400, 600, 3)))

True

In [30]:
nbytes = len(bytes)
if nbytes % 8 != 0: print('corrupted file')

if channels == 3:
    img_line = np.empty((0, 3), dtype=np.uint8)
elif channels == 4:
    img_line = np.empty((0, 4), dtype=np.uint8)

if channels == 3:
    pixel_prev = np.array([0, 0, 0], dtype=np.uint8)
elif channels == 4:
    pixel_prev = np.array([0, 0, 0, 255], dtype=np.uint8)
else: print('error: wrong number of channels')

memory = {i: np.array([0, 0, 0]) for i in range(64)}
i = 0
while i < nbytes - 8:

    chunk = bytes[i:i+8]
    
    # Original value (RGB)
    if channels == 3 and chunk == b'\x01\x01\x01\x01\x01\x01\x01\x00':
        # print('new')
        r = bytes[i+8:i+16]
        g = bytes[i+16:i+24]
        b = bytes[i+24:i+32]
        r_int = int.from_bytes(r, byteorder=BYTEORDER, signed=False)
        g_int = int.from_bytes(g, byteorder=BYTEORDER, signed=False)
        b_int = int.from_bytes(b, byteorder=BYTEORDER, signed=False)
        pixel = np.array([r, g, b], dtype=np.uint8)
        # memory[index_hash(pixel)] = pixel
        img_line = np.concatenate((img_line, np.expand_dims(pixel, axis=0)))
        i += 24
    # Original value (RGBA)
    elif channels == 4 and chunk == b'\x01\x01\x01\x01\x01\x01\x01\x01':
        # print('new')
        r = bytes[i+8:i+16]
        g = bytes[i+16:i+24]
        b = bytes[i+24:i+32]
        a = bytes[i+32:i+40]
        r_int = int.from_bytes(r, byteorder=BYTEORDER, signed=False)
        g_int = int.from_bytes(g, byteorder=BYTEORDER, signed=False)
        b_int = int.from_bytes(b, byteorder=BYTEORDER, signed=False)
        a_int = int.from_bytes(a, byteorder=BYTEORDER, signed=False)
        pixel = np.array([r, g, b, a], dtype=np.uint8)
        # memory[index_hash(pixel)] = pixel
        img_line = np.concatenate((img_line, np.expand_dims(pixel, axis=0)))
        i += 32

    # Run
    elif chunk[:2] == b'\x01\x01':
        # print('run')
        run_len = int.from_bytes(chunk[2:], byteorder=BYTEORDER, signed=False)
        pixels = np.full((run_len + 1, channels), pixel_prev, dtype=np.uint8)  # +1 here or encode run_len+1?
        img_line = np.concatenate((img_line, pixels))

    # Big dif
    elif chunk[:2] == b'\x01\x00':
        # print('bigdif')
        dg = int.from_bytes(chunk[2:], byteorder=BYTEORDER, signed=True)
        dr = int.from_bytes(bytes[i+8:i+12], byteorder=BYTEORDER, signed=True) + dg
        db = int.from_bytes(bytes[i+12:i+16], byteorder=BYTEORDER, signed=True) + dg
        if channels == 3: pixel_diff = np.array([dr, dg, db])
        elif channels == 4: pixel_diff = np.array([dr, dg, db, 0])
        pixel = (pixel_prev.copy() + pixel_diff).astype(np.uint8)
        # memory[index_hash(pixel)] = pixel
        img_line = np.concatenate((img_line, np.expand_dims(pixel, axis=0)))
        i += 8
    
    # Small dif
    elif chunk[:2] == b'\x00\x01':
        # print('smalldif')
        dr = int.from_bytes(chunk[2:4], byteorder=BYTEORDER, signed=True)
        dg = int.from_bytes(chunk[4:6], byteorder=BYTEORDER, signed=True)
        db = int.from_bytes(chunk[6:8], byteorder=BYTEORDER, signed=True)
        if channels == 3: pixel_diff = np.array([dr, dg, db])
        elif channels == 4: pixel_diff = np.array([dr, dg, db, 0])
        pixel = (pixel_prev.copy() + pixel_diff).astype(np.uint8)
        # memory[index_hash(pixel)] = pixel
        img_line = np.concatenate((img_line, np.expand_dims(pixel, axis=0)))

    # Index
    elif chunk[:2] == b'\x00\x00':
        # print('memory')
        # mk = list(memory.keys())
        # mk.sort()
        # print(i, mk)
        print(chunk)
        index = int.from_bytes(chunk[2:], byteorder=BYTEORDER, signed=False)
        pixel = memory[index].copy()
        # memory[index_hash(pixel)] = pixel
        img_line = np.concatenate((img_line, np.expand_dims(pixel, axis=0)))
    
    # Add pixel to memory
    # print(index_hash(pixel), pixel)
    memory[index_hash(pixel)] = pixel
    pixel_prev = pixel.copy()

    i += 8

print(img_line)

corrupted file
b'\x00\x00\xfe#?u\xfec'


KeyError: 279427342007907

In [187]:
u = np.array([1, 2, 3])
v = np.array([0, 0, 0])
print(v.shape)

a = np.empty((0, 3), dtype=np.uint8)
print(a)
b = np.full((10, 3), u)

(3,)
[]


In [205]:
l = np.concatenate((a, b))

In [204]:
np.concatenate((l, np.expand_dims(v, axis=0)))

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [0, 0, 0]])

In [268]:
u = np.array([10, 28, 31], dtype=np.uint8)

In [269]:
print(u)

[10 28 31]


In [273]:
26*u

array([  4, 216,  38], dtype=uint8)

In [66]:
u = '11{0:06b}'.format(17)
print(u)

ub = int(u, 2).to_bytes(1, byteorder=BYTEORDER, signed=False)
print(ub)

11010001
b'\xd1'


In [39]:
int.from_bytes(b'\x00\x00\x00\x00\x01\x00', byteorder=BYTEORDER, signed=False)

256

In [75]:
r = -1
'{0:02b}'.format(r + 2)

'01'

In [88]:
dg = -30
dr_dg = 7
db_dg = 3

vals = ['10' + '{0:06b}'.format(dg+32), '{0:04b}'.format(dr_dg+8) + '{0:04b}'.format(db_dg+8)]
vals_b = [int(val, 2).to_bytes(1, byteorder=BYTEORDER, signed=False) for val in vals]
print(vals_b)
vals_b[0] + vals_b[1]


[b'\x82', b'\xfb']


b'\x82\xfb'